

# Publikovanie elektrických schém a diagramov

<img src="./src_circ/circ_10.png" width=550px alt="Title" scale="0.6"/>

## Úvod

Denným chlebom každého technika je kreslenie schém a diagramov, pričom prakticky v každej oblasti existujú vhodné programy, ktoré túto činnosť zjednodušujú a uľahčujú. Pri rutinej činnosti je kladený dôraz na obsah, forma často nie je podstatná, dôležité je hlavne to, aby diagram prehľadne zobrazoval to, k čomu bol vytvorený. Pri potrebe publikovania schém a diagramov v článkoch a knihách je ale okrem obsahu často dôležitá aj forma, ako schéma alebo diagram vyzerá, či dodržiava stanovené normy a interné pravidlá. V dávnych dobách stačilo zaslať do redakcie článok s ručne nakreslenou schémou a kresliči v redakcii zapojenie prekreslili podľa štýlu časopisu, starší čitatelia si zaiste nostalgicky spomenú na precízne ručne kreslené zapojenia v Sdělovací technike, Slaboproudem obzoru a ďalších.   

Súčasná doba už kresleniu tušom na pauzák nepraje, na druhej strane bežné programy zvyčajne nie sú orientované na generovanie výstupov do publikácií, aj keď odborné publikácie so schémami vytvorenými v Eagle nie sú žiadnou zvláštnosťou. Väčšie problémy nastanú, ak potrebujeme schému alebo diagram doplniť ďaľšími grafickými prvkami - matematickými vzorca, šípkami, doplnkovými grafmi a pod. Pre jednorázové nakreslenie schémy zvyčajne splní všetky požiadavky grafický, zvyčajne vektorový editor (Inkscape, sk1 ..). Pokiaľ ale potrebujeme obrázky modifikovať, vkladať do nich matematické výrazy, parametrizovat alebo možno aj programovo generovať, jednou z možností je použitie v súčasnej dobe možno trochu archaického spôsobu vytvárania schém a diagramov pomocou textového popisu - makier.


## Kreslenie schém a diagramov pomocou circuit-macros

Z pohľadu súčasného uživateľa rozmaznaného klikacími programami môže pripadať v súčasnej dobe kreslenie zapojení a diagramov pomocou textových makier niečo ako použitie parného stroja v športovom aute. Pri podrobnejšom pohľade na princíp vytvárania zapojení pomocou textového popisu zistíme, že princíp sa nie veľmi odlišuje od toho, čo v súčasnej dobe poznáme pod pojmom parametrické modelovanie.

<img src="./src_circ/demo.png" width=650px alt="Nostalgická ukážka" scale="0.4"/>


Knižnica circuit-macros (CM) [1] obsahuje všetky bežné komponenty potrebné pre vytváranie elektrických zapojení v tvare makier pre makroprocesor m4 [2]. Pomocou makroprocesora sa popis zapojenia preloží do jazyka PIC [3], ktorý je určený pre všeobecnú tvorbu grafiky a jeho autorom je legenda IT - Brian W. Kernighan. Pomocou kompilátora dpic [3] sa grafický popis preloží do cielového formátu (svg, ps, pdf ...) alebo do makier PSTricks pre priame použitie v LaTex-u. Uvedený postup vyzerá komplikovaný, ale jednoduchý skript dokáže všetky činnosti automatizovať. 

### Inštalácia

Inštalácia circuit-macros pozostáva zo stiahnutia archívu z domácej stránky a rozbalení do adresára. Distribúcia obsahuje sadu makier a príkladov, neobsahuje žiaden vykonateľný program.

Makroprocesor m4 je štandardnou súčasťou linuxových distribúcií. 

Kompilátor dpic je súčasťou repozitárov

    sudo apt-get install dpic


### Štruktúra zdrojového textu

Typický zdrojový text jednoduchého zapojenia v CM má tvar

    .PS                   
    cct_init           
    scale = 2.54     
    d=1.5               
    resistor(right_ d, E); llabel(,R_1,); 
    resistor(down_  d, E); llabel(,R_2,); 
    .PE            
    
* *.PS* a *.PE* označujú začiatok a koniec skriptu
* *cct_init* je volanie inicializacie skriptu a premenných
* *scale=2.54* je nastavenie mierky kreslenia pre mierku 1cm, default mierka je 1 inch   
* *d=1.5* je premenná, ktorá bude použitá ako dĺžkový parameter 

<img src="./src_circ/circ_05.png" width=150px alt="Zapojenie z príkladu" scale="1.0"/>

Súradníce sú v karteziánskej sústave. Pri začiatku kreslenia je fiktívne pero nastavené do polohy (0,0). Definovanie komponentu znamená jeho vykreslenie a presun pera do novej polohy, ktorá je definovaná v tomto komponente. 

Základným grafickým prvkom je orientovaný dvojpól, zoznam všetkých typov komponentov je uvedený v dokumentácii [4]. Postupnosť príkazov

    resistor(right_ d, E); 
    llabel(,R_1,);

uloží na vykres rezistor orientovany doprava (*right_*) o dĺžke 1.5cm (*d*) v tvare podľa európskej normy (*E*). Na lavú stranu rezistora v smere ukladania (*llabel*) umiestni označenie rezistora. Pretože predpokladáme, že budeme  generovať výstupný kód pomocou PSTricks, budú označenie renderované ako matematické vzťahy v LaTex-e. Po vykreslení rezistoru sa pero presunie na koniec rezistoru a príkaz

    resistor(down_  d, E); llabel(,R_2,);
    
vykreslí nový rezistor v definovanom smere. Na tomto mieste treba upozorniť na používanie parametrov, je rozdiel medzi down_ a down (bez podtržítka), prvý tvar používa CM, pri ktorom nastavuje aj parametre iných vnútorných premenných napr. pre polohu označovania komponentov, druhý tvar štandardne používa PIC. Z princípu fungovania makroprocesora *m4* vyplýva, že ak pri preklade narazí na neznámu konštrukciu, tak ju ponechá v pôvodnom stave (down). Na druhej strane, na základe tohoto môžeme pri kreslení zároveň používať príkazy jazyka PIC pre kreslenie v zapojeniach.

### Preklad a kompilácia

Najprv uvedieme komplikovanejší postup s využitím renderovania textov v LaTex-e a s využitím makier PSTricks, predpokladáme, že LaTex s príslušnými modulmi máte už nainštalovaný. V prvom kroku preložíme naše zapojenie pomocou makroprocesora do príkazov jazyka PIC

    m4 -I PATH_TO_CM_MACROS pstricks.m4 SOURCE_FILE > PIC_FILE

v príkaze zadefinujeme cestu k adresáru, kde sme rozbalili makrá z archívu, náš zdrojový súbor so zapojením a zvolíme meno súboru, do ktorého sa uložia príkazy v PIC. *pstricks.m4* obsahuje sadu pomocných makier pre finálny výstup. 

V druhom kroku súbor v jazyku PIC do sady makier PSTricks preložíme pomocou

    dpic -p PIC_FILE > PSTRICK_FILE

Výsledkom prekladu je zdrojový text pre LaTexu-u s príkazmi pre PSTricks, ktorý má štandardný tvar

    \begin{pspicture}
    ...
    \end{pspicture}

Skompilovaný súbor už môžeme vložiť do dokumentu v LaTex-u a ďalej s ním pracovať. Pretože pri kreslení schém potrebuje priebžne sledovať výsledok práce, pre renderovanie obrázku zapojenia do postscriptu môžeme použiť krátky dokument (LATEX_FILE)

    \documentclass{article}
    \usepackage{times,pstricks,pst-eps,pst-grad}
    \usepackage{graphicx}
    \begin{document}
    \begin{TeXtoEPS}
    
    \input PSTRICK_FILE        <- menu skompilovaneho suboru bez pripony
    
    \end{TeXtoEPS}\end{document}

ktorý preložíme 

    latex LATEX_FILE      <- bez pripony .tex
    dvips -E LATEX_FILE   <- bez pripony .aux
    
výsledkom prekladu je obrázok vo formáte postscript. Aj keď uvedený postup vyzerá komplikovane, jednoduché skripty v pythone alebo shell-e problém vyriešia. 

Pri jednoduchšom spôsobe grafický súbor vygeneruje priamo kompilátor *dpic*, samozrejme ale bez renderovania textov LaTex-om.

    m4 -I PATH_TO_CM_MACROS postscript.m4 SOURCE_FILE | dpic -r > PS_FILE


## Poznámky ku kresleniu schém

Úlohou tohoto príspevku nie je opisovať manuály [4][5], s pomocou príkladov v distribúcii ako aj ukážkami vytvorenými nadšencami po internete [6] si je možné si potrebné postupy pri kreslení zapojení rýchlo osvojiť. Ukážeme len niektoré detaily, ktoré by mohli spôsobiť na začiatku drobné problémy

### Referencie

Na vytvorené objekty je možné sa odkazovať menom, napr.

    R1: resistor(right_ d, E); 
    
a ďaľšom potom môžeme využívať jeho atribúty ako súradnice, pre dvojpóly sú definované napr.

    R1.start
    R1.center
    R1.end
    R1.dimen_
    ...

### Súradnice a vetvenie

Aktuálna poloha je definovaná premenou *Here*. Absolútne aj relatívne súradnice môžeme používat v tvare (x,y),
pre určenie smeru *right_, left_, up_, down_*, ktoré môžeme aj kombinovať. Menom sa môžeme odkazovať na súradnice častí objektov

    R.end   <->  (R1.end.x, R1.end.y)

Nasledujúce zápisy sú rovnocenné

    d=1.5;
    resistor from Here to Here + (d,d);
    resistor to Here + (d,d);
    resistor to Here + (1.5, 1.5);
    resistor( right_ d down_ d); 

Pri vetvení obvodov aktuálny polohu odložime tým, že vetvu uzatvoríme do zložených zátvoriek. Pre vykreslenie elektrického spojenia slúži makro *dot*.         

    R1: resistor(right_ d, E);   llabel(,R_1,);
    dot;
    {   # odlozenie aktualnej polohy a vykreslenie vetvy
        R2: resistor(up_ d, E);  llabel(,R_2,);
        C1: capacitor(right_ d); llabel(,C_1,);
    }   
    # obnovenie aktualnej polohy 
    R3: resistor(down_ d, E); llabel(,R_3,);
    L1: inductor(right_ d);   llabel(,L_1,);

<img src="./src_circ/circ_01.png" width=180px alt="Vytvorenie vetvy obvodu" scale="1.0"/>

### Prepojenie medzi komponentami

Pre vytvorenie spojov slúžia makro *line from to*, *line to*. Pre presun bez kreslenia na novú pozíciu je určené makro *move to*. K predchádzajúcemu príkladu doplníme 

    line from C1.end to (C1.end, R1.end);  # ekvivalent (C1.end.x, R1.end.y)
    dot;
    {   # vetva
        R4: resistor(right_ d, E); llabel(,R_4,);
    }
    line to L1.end
    
<img src="./src_circ/circ_02.png" width=280px alt="Vytvorenie prepojenia" scale="1.0"/>

### Zmena polarity komponentu

Dvojpóly sú orientované komponenty, ktoré začínajú na pozícii *start* a končia na pozícii *end*. Pri polarizovaných komponentoch (napr. dióda, elektrolytický konddenzátor) je niekedy potrebné ho nakresliť v opačnej polarite. Pre zmenu polarity komponentu je možné použiť makro *reversed*, ktorému predáme názov makra komponentu a jeho argumenty. Meno makra je v uvodzovkách podľa syntaxe *m4*, kde ako **otváracia uvodzovka** je použitý znak \` - back quote a **zatváracia úvodzovka** je ' apostrof resp. single quote

    dot;
    {   # prava cast mostika
        D1: diode(down d/sqrt(2) right d/sqrt(2) ); llabel(,,D_1)
            dot;
        D2: reversed(`diode', down d/sqrt(2) left d/sqrt(2) ); rlabel(,,D_2)
            dot;
    }
    {   # lava cast mostika
        D3: reversed(`diode', down d/sqrt(2) left d/sqrt(2) ); llabel(D_3,,)
            dot;
        D4: diode(down d/sqrt(2) right d/sqrt(2) ) ; rlabel(D_4,,)

    }

<img src="./src_circ/circ_03.png" width=150px alt="Zmena polarity komponentov" scale="1.0"/>

Pretože CM je nadstavbou jazyka PIC, je možné samozrejme využívať aj všetky jeho konštrukcie, vytváranie vlastných makier, cykly, podmienky a mnohé ďaľšie. Podrobnosti si každý záujemca nájde v manuáloch.

## Záver

Okrem uvedených metód kreslenia zapojení a diagramov do publikáciíí existuje aj ďalšie možnosti, napr. prostredie CircuiTikz pre LaTex. Predmetom článku bola krátka prezentácia knižnice makier circuit-macro, nie je možné v rámci článku ukázať všetky možnosti a zároveň ani nechceme pripraviť čitateľov o radosť z objavovania veľmi nákazlivého spôsobu vyjadrenia vlastných ideí.
Aj napriek na prvý pohľad zastaralému a zdĺhavému postupu si môže circuit-macros nájsť svoje miesto pod slnkom - od jednoduchého kreslenia schém do publikácií až po programovo generované schémy a diagramy napr. pre oblasť výučby na tvorbu personalizovaných príkladov a písomiek pomocou parametricky zadaných podkladov (za toto ma niekto nebude mať rád), čo je v súčasnej dobe aktuálna téma.

## Literatúra

[1] https://ece.uwaterloo.ca/~aplevich/Circuit_macros/

[2] https://en.wikipedia.org/wiki/M4_(computer_language)

[3] https://gitlab.com/aplevich/dpic

[4] https://ece.uwaterloo.ca/~aplevich/Circuit_macros/Circuit_macros.pdf

[5] http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.510.6190&rep=rep1&type=pdf

[6] http://ytdp.ee.wits.ac.za/cct.html

http://ytdp.ee.wits.ac.za/help/gpic.pdf


http://floppsie.comp.glam.ac.uk/Glamorgan/gaius/web/pic.html








